In [2]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [17]:
# Empty Merkle Block example
import math

from helper import merkle_parent

total = 1833
max_depth = math.ceil(math.log(total, 2))
hashes = [bytes.fromhex(h) for h in hex_hashes]
merkle_tree = []
for depth in range(max_depth+1):
    num_items = math.ceil(total / 2**(max_depth - depth))
    level_hashes = [None] * num_items
    merkle_tree.append(level_hashes)

4dbf3ade1ed6595de5f9b4ab4a17a80d4bcc53227ac0b0eedbb237b595a73737


In [ ]:
flag_bits = [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
hex_hashes = [
    '96979db23ca3f0dbf3e4d748b3e845cfbed09c843967374969e1b703cd358458',
    'c9f0112d8a7fbb3b7364f14a5d0ba8e311a918dc735d9933df7912782bbf82fe',
    '6e6ce882f409e99d5763fd6afa02fbabb4b5f047ef0969ba6619a69fcc54d2c3',
    'c797c2248b270b28f35b530aefbaa0c5edb4f12343b44def58382496a93e8699',
    '46b4bdbe6ea1a845abff2bf36f6f39507c7eea292312bee36bc4bd96363871e9',
    '5a5b5f1268eb00dd47046a115a228d3c76cf0414bedf4e187b37918d5715bfb6',
    '8a0b9f69baaec595a7d9c00d1443f721ade27e99e4190511e1b24812d3f964d6',
    'd03d6f5cb9c308a2b795d44e1638a7aa5d0eb59e9c4eb5c313aca49ed3eb2619',
    '49e035addfe3c8cfc264eb34ebc9ad40609d9d53a03f3a7929f81dbf4f2e974f',
    '15ba5e2a376d9b269c1b99eec3d037c8228b25e33142aed6c00f2fedce8027db',
    '30fad74273b6ed865deae7eb30158d9a0fc1459e6d212a91c865ccaf59092d74',
    '1d09dd247812fcf2a1f95e31947abc3184f836aab904860ede28d4a5a072033c',
]
current_depth, current_index = 0, 0
while merkle_tree[0][0] is None:
    if current_depth == max_depth:
        current_bit = flag_bits.pop(0)
        merkle_tree[current_depth][current_index] = hashes.pop(0)
        next_depth, next_index = current_depth - 1, current_index // 2
    else:
        left_index = current_index * 2
        right_index = left_index + 1
        left_hash = merkle_tree[current_depth+1][left_index]
        if left_hash is None:
            if flag_bits.pop(0) == 0:
                merkle_tree[current_depth][current_index] = hashes.pop(0)
                next_depth, next_index = current_depth - 1, current_index // 2
            else:
                next_depth, next_index = current_depth + 1, left_index
        else:
            if right_index >= len(merkle_tree[current_depth+1]):
                merkle_tree[current_depth][current_index] = merkle_parent(left_hash, left_hash)
                next_depth, next_index = current_depth - 1, current_index // 2
            elif merkle_tree[current_depth+1][right_index]:
                right_hash = merkle_tree[current_depth+1][right_index]
                merkle_tree[current_depth][current_index] = merkle_parent(left_hash, right_hash)
                next_depth, next_index = current_depth - 1, current_index // 2
            else:
                next_depth, next_index = current_depth + 1, right_index
    current_depth, current_index = next_depth, next_index
print(merkle_tree[0][0].hex())

In [28]:
reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_is_valid'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 1

1. Given a Bloom Filter with these parameters: size=20, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

2. Make this test pass.

In [13]:
# Exercise 1.1

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import murmur3

size = 10
function_count = 5
tweak = 99

items = (b'Hello World',  b'Goodbye!')

# first the actual filter which is a bytearray of size size
f = bytearray(size)

# for each item you want to add to the filter
for item in items:
    # iterate function_count number of times
    for i in range(function_count):
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        h = murmur3(item, seed=seed)
        # set the bit at the hash mod the total number of bits in the filter
        bit = h % (size * 8)
        # get the byte index and bit index
        filter_index, bit_index = divmod(bit, 8)
        # the filter's byte index needs the bit at the bit_index set
        f[filter_index] |= (1 << bit_index)
    print(bytes(f).hex())

0000000a080000000140
4000600a080000010940


In [17]:
# Exercise 1.2

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_add'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 2

1. Make this test pass.

In [19]:
# Exercise 2.1

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_filterload'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [24]:
reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_is_valid'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [ ]:
reload(tx)
reload(network)
reload(bloomfilter)
reload(merkleblock)

import time

from random import randint
from io import BytesIO

from block import Block
from bloomfilter import BloomFilter
from helper import double_sha256, int_to_little_endian, encode_varint, read_varint, decode_base58, p2pkh_script, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import NetworkEnvelope, SimpleNode, GetHeadersMessage
from tx import Tx, TxIn, TxOut
from ecc import PrivateKey

private_key = PrivateKey(secret=int.from_bytes(double_sha256(b'Jimmy Song'), 'big'))

addr = 'mtShBfBH9LFyHmqJFuKjmAPfv6jJmwpbeV'
h160 = decode_base58(addr)
bf = BloomFilter(30, 6, 42)
bf.add(h160)

target_address = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 50000  # fee in satoshis

node = SimpleNode('tbtc.programmingblockchain.com', testnet=True)

# complete the handshake
node.handshake()
node.send(b'filterclear',b'')
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())

# ask for blocks since block X
starting_block = bytes.fromhex('0000000000000020ec00e9a5e463d9d61ca848c59ad1f37cc671bf8775c4884f')
getheaders_message = GetHeadersMessage(starting_block=starting_block)
node.send(b'getheaders', getheaders_message.serialize())

# wait for headers
headers_envelope = node.wait_for_commands({b'headers'})

# loop through each header and ask for the merkleblock
stream = BytesIO(headers_envelope.payload)
num_headers = read_varint(stream)
payload = encode_varint(num_headers)
last_block = None
for _ in range(num_headers):
    b = Block.parse(stream)
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    num_txs = read_varint(stream)
    if num_txs != 0:
        raise RuntimeError('got more than 0 txs')
    if last_block is not None and b.prev_block != last_block:
        raise RuntimeErrer('chain broken')
    # ask for merkleblock
    payload += int_to_little_endian(3, 4)
    payload += b.hash()[::-1]
    # set the last block
    last_block = b.hash()
node.send(b'getdata', payload)

# look for merkleblock or tx commands and validate them
txos = {}
count = 0
while True:
    envelope = node.wait_for_commands({b'merkleblock', b'tx'})
    stream = BytesIO(envelope.payload)
    if envelope.command.startswith(b'merkleblock'):
        continue
        mb = MerkleBlock.parse(stream)
        if not mb.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        count += 1
        t = Tx.parse(stream, testnet=True)
        for tx_in in t.tx_ins:
            key = '{}:{}'.format(tx_in.prev_tx.hex(), tx_in.prev_index)
            if txos.get(key):
                txos.pop(key)

        for i, tx_out in enumerate(t.tx_outs):
            try:
                if tx_out.hash160() == h160:
                    txos['{}:{}'.format(t.hash().hex(),i)] = t
            except:
                pass
        if count == 3:
            break
# create tx_ins
tx_ins = []
total = 0
for key, pt in txos.items():
    _, prev_index = key.split(':')
    prev_index = int(prev_index)
    tx_ins.append(TxIn(pt.hash(), prev_index, b'', 0xffffff))
    total += pt.tx_outs[prev_index].amount
    
tx_outs = [TxOut(total-fee, target_script)]
t = Tx(1, tx_ins, tx_outs, 0, testnet=True)
for i in range(len(tx_ins)):
    t.sign_input(i, private_key, SIGHASH_ALL)
print(t.serialize().hex())
# first, add this transaction to our bloom filter
# send this signed transaction on the network
node.send(b'tx', t.serialize())
# now wait for a response
envelope = node.wait_for_commands({b'inv', b'reject'})
print(envelope.command)
print(envelope.payload.hex())